In [12]:
import pandas as pd


Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [13]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd = rzd.groupby('client_id').sum()# группирую по столбцу client_id, чтобы исключить повторения
#  складываю значения, перевела в индекс столбец client_id
rzd.rename(columns={'rzd_revenue': 'rzd'},inplace=True)
rzd

,rzd
client_id,
111,1093
112,2810
113,10283
114,5774
115,981


In [14]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto = auto.groupby('client_id').sum()
auto.rename(columns={'auto_revenue': 'auto'},inplace=True)
auto

,auto
client_id,
113,57483
114,83
115,912
116,4834
117,98


In [15]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air = air.groupby('client_id').sum()
air.rename(columns={'air_revenue': 'air'},inplace=True)
air

,air
client_id,
115,81
116,4
117,13
118,173


In [16]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base = client_base.groupby('client_id').sum()
client_base

,address
client_id,
111,Комсомольская 4
112,Энтузиастов 8а
113,Левобережная 1а
114,Мира 14
115,ЗЖБИиДК 1
116,Строителей 18
117,Панфиловская 33
118,Мастеркова 4


In [17]:
total = (
    rzd
    .join(auto, how='outer')
    .join(air, how= 'outer')
)
total

,rzd,auto,air
client_id,,,
111,1093.0,NaN,NaN
112,2810.0,NaN,NaN
113,10283.0,57483.0,NaN
114,5774.0,83.0,NaN
115,981.0,912.0,81.0
116,NaN,4834.0,4.0
117,NaN,98.0,13.0
118,NaN,NaN,173.0


In [18]:
total_address = total.join(client_base, how='outer')
total_address

,rzd,auto,air,address
client_id,,,,
111,1093.0,NaN,NaN,Комсомольская 4
112,2810.0,NaN,NaN,Энтузиастов 8а
113,10283.0,57483.0,NaN,Левобережная 1а
114,5774.0,83.0,NaN,Мира 14
115,981.0,912.0,81.0,ЗЖБИиДК 1
116,NaN,4834.0,4.0,Строителей 18
117,NaN,98.0,13.0,Панфиловская 33
118,NaN,NaN,173.0,Мастеркова 4


## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. 
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок.
Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [19]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [20]:
#ratings.head()

In [21]:
len(ratings)

100004

In [22]:
user_100 = (
    ratings
    .filter(items = ['userId', 'rating'])# только нужные столбцы
    .groupby('userId').count()#отсортировала по юзерам частоту
    .query('rating > 100')# беру тех кто  > 100
    
)

In [23]:

user_100.head()

,rating
userId,
4,204
8,116
15,1700
17,363
19,423


In [24]:
len(user_100)

258

In [25]:
user_100.rename(columns={'rating': 'rating_100'},inplace=True)

In [26]:
user_100.rating_100.sort_values(ascending=False).head()


userId
547    2391
564    1868
624    1735
15     1700
73     1610
Name: rating_100, dtype: int64

In [27]:
user_100.reset_index(inplace=True)

In [28]:
user_100.head()


,userId,rating_100
0,4,204
1,8,116
2,15,1700
3,17,363
4,19,423


In [29]:
list_user_100 = user_100['userId'].tolist()
len(list_user_100)

258

In [30]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [31]:
ratings = ratings[ratings['userId'].isin(list_user_100)]
    

In [32]:
ratings.head()

,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346


In [33]:
len(ratings)

80487

In [34]:
ratings = (
    ratings
    .filter(items = ['userId', 'timestamp'])# только нужные столбцы  
    .groupby('userId')
    .agg([min, max])

)

In [35]:
ratings.head()

timestamp            
               min         max
userId                        
4        949778714   949982274
8       1154389340  1154474527
15       997937239  1469330735
17      1127468587  1127476640
19       855190091   855195373

In [36]:
len(ratings) # совпадает с длиной list_user_100

258

In [37]:
ratings['diff']= ratings['timestamp']['max']  - ratings['timestamp']['min']
ratings.head()

timestamp                   diff
               min         max           
userId                                   
4        949778714   949982274     203560
8       1154389340  1154474527      85187
15       997937239  1469330735  471393496
17      1127468587  1127476640       8053
19       855190091   855195373       5282

In [38]:
 #    КРУТЬ!!! 
ratings['diff'].sort_values(ascending=False).head()

userId
380    515789661
547    501810535
427    476569256
15     471393496
624    457492507
Name: diff, dtype: int64

In [152]:
501810535/31536000
# пользователь 547 почти 16 лет ставил  по 150 оценок в год!! 
2391/16

149.4375

## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Как-то совсем абстрактно мне сложно рассуждать, я это попробую сделать через призму компании, которая сдает машины в аренду ( думаю, что им  наличе коордитат пользователя просто необходимо). 
Вопрос 1 - я бы в столбец заносила не широту/ долготу, а районы(области, которые заранее были бы определены: в таком диапозоне широты и долготы такой район). Таким образом был бы создан датафрейм со столбцами: пользователь, визит, покупка, месторасположение
Вопрос 2 - в моем случае это бы не имело знаение
Вопрос 3 - проанализировав дф можно было выяснить в каких местах есть большой спрос , но нет покупок, возможно там необходимо сделать парковки машин. 

Если расссматривать в общем, то в зависимости от анализа перемещения пользователя, можно формировать ему предложения : покупку для авто, домохозяйки что-то по дому, если мы видим что он активен когда по линии метро- предложим ему защитные средства от вирусов! 

Видимо я совсем не по заданию, но вот так! 